In [1]:
from transformers_neuronx.llama.model import LlamaForSampling
from transformers import AutoTokenizer
import torch
import time
import os 
import torch_neuronx
import sys 


/home/ubuntu/anaconda3/envs/inf2.15.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['NEURON_RT_NUM_CORES'] = '2'
model_dir = "../2.15.1/model_store/llama-2-7b-chat/llama-2-7b-chat-split"
model_compiled_dir="../2.15.1/model_store/llama-2-7b-chat/neuronx_artifacts"

In [3]:
start = time.time()
model = LlamaForSampling.from_pretrained(
        model_dir,
        batch_size=1,
        tp_degree=int(os.environ['NEURON_RT_NUM_CORES']),
        amp='f16',
        )
model.load(model_compiled_dir)
model.to_neuron()
elapsed = time.time() - start
print(f'\n Model successfully loaded in {elapsed} seconds')


 Model successfully loaded in 110.05396628379822 seconds


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.add_special_tokens(
        {
        
            "pad_token": "<PAD>",
        }
    )

1

In [33]:
prompt = "Explain the concept of Generative AI to me as if I am an 8 year old child. \n Response:"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

payload = {
   "inputs": "Explain the concept of Generative AI to me as if I am an 8 year old child. \n Response:",
    "parameters": {'max_new_tokens': 50, 'top_p': 0.9, 'temperature': 0.6}
}

# run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = model.sample(input_ids, 
                                        sequence_length=128, 
                                        top_k=50, 
                                        temperature=0.5)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq) for seq in generated_sequences]


In [34]:
generated_sequences

["<s> Explain the concept of Generative AI to me as if I am an 8 year old child. \n Response:\n Generative AI is like a magic machine that can make new and exciting things! Imagine you have a toy box full of blocks, and you can use those blocks to build anything you want - a castle, a car, a robot! But with Generative AI, the machine can build things that you can't even imagine! It can make new and exciting things that you've never seen before, like a dragon or a spaceship.\n\nThe machine"]

In [35]:
response = generated_sequences[0].strip('<s> ')[len(prompt):]
print(len(response))
print(response)

397

 Generative AI is like a magic machine that can make new and exciting things! Imagine you have a toy box full of blocks, and you can use those blocks to build anything you want - a castle, a car, a robot! But with Generative AI, the machine can build things that you can't even imagine! It can make new and exciting things that you've never seen before, like a dragon or a spaceship.

The machine


In [54]:
import json
payload = {
   "inputs": "Explain the concept of Generative AI to me as if I am an 8 year old child. \n Response:",
    "parameters": {'max_new_tokens': 100, 'top_p': 0.9, 'temperature': 0.6}
}

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = model.sample(input_ids, 
                                        sequence_length=payload['parameters']['max_new_tokens']+len(payload['inputs']), 
                                        top_p=payload['parameters']['top_p'], 
                                        temperature=payload['parameters']['temperature'])
    elapsed = time.time() - start
print(f'Inference took {elapsed} seconds')
generated_sequences = [tokenizer.decode(seq) for seq in generated_sequences]
response = [
    {
        "generation":generated_sequences[0].strip('<s> ')[len(prompt):]
    }
]
print(json.dumps(response, indent=2))

Inference took 9.626596212387085 seconds
[
  {
    "generation": "\n Generative AI is like a magic machine that can make new things that never existed before! It's like a super cool toy that can create anything you can imagine.\n\nImagine you have a big box of LEGOs, and you want to build a new toy car. You can use the LEGOs you have to build the car, but the machine that makes the car can also make new shapes and designs that you never thought of before! It's like the machine has its own imagination and can create new things that are even cooler than what you can make.\n\nThat's what generative AI is like! It's a machine that can make new things, like images, music, or even whole new games, just"
  }
]
